In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
import binarytree

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [17]:
class Arguments():
    def __init__(self):
        self.images = 50000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [18]:
writer = open('20_experiments_cifar.csv', 'a')

In [19]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [20]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [21]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [23]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
global_test_dataset = datasets.CIFAR10('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

Files already downloaded and verified


In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [25]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [26]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [27]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for client in all_models:
        all_dicts[client] = all_models[client].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [28]:
def BtreeAvg(root):
    global global_model
    temp_model = Aggregate({'root':root.value[1], 'global':global_model}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvg(root.left)
    if root.right:
        right = BtreeAvg(root.right)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [29]:
def BtreeAvgSecond(root, final):
    temp_model = Aggregate({'root':root.value[1], 'global':final}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvgSecond(root.left, temp_model)
    if root.right:
        right = BtreeAvgSecond(root.right, temp_model)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [30]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

tree_List = []
for inds, i in enumerate(clients):
    temp = [None] * 3
    temp[0] = i
    temp[1] = clients[inds]['model']
    temp[2] = clients[inds]['hook'].id
    tree_List.append(temp)

root = None
root = binarytree.insertLevelOrder(tree_List, root, 0, len(tree_List))
# root.printTree()

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
        
#     global_model = averageModels(global_model, active_clients)
#     test(args, global_model, device, global_test_loader, 'Global')
#     normal_avg = normalModels(normal_avg, active_clients)
#     test(args, normal_avg, device, global_test_loader, 'Global')
    
    
    # Begin the Btree
#     final_model = BtreeAvg(root)
    final_model = BtreeAvgSecond(root, global_model)
#     glo_model = copy.deepcopy(final_model)
    global_model.load_state_dict(final_model.state_dict())
#     test(args, global_model, device, global_test_loader, 'Global')
#     break
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
#     # Averaging 
#     global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

Model client16 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.323728
Model client16 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.300640
Model client16 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.319030
Model client16 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.310430
Model client16 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.302511
Model client2 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.322279
Model client2 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.292531
Model client2 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.299138
Model client2 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.310438
Model client2 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.306267
Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.307345
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.318027
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.306035
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.301256
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.298143
Model client5 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.300318
Model client5 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.306651
Mode

Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.293911
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.302959
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.299927
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.302125
Model client18 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.301906
Model client18 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.301348
Model client17 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.314265
Model client17 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.314973
Model client17 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.301885
Model client17 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.306773
Model client17 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.304549
Model client19 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.299448
Model client19 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.297537
Model client19 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.297345
Model client19 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.294991
Model client19 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.306365
Model client4 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.29748

Model client4 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.297092
Model client4 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.295283
Model client9 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.297648
Model client9 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.304114
Model client9 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.299039
Model client9 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.302661
Model client9 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.298096
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.302503
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.305349
Model client3 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.292259
Model client3 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.298103
Model client3 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.291237
Model client7 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.300250
Model client7 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.307550
Model client7 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.299107
Model client7 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.304843
Model client7 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.300418
Model client2 

Model client1 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.276469
Model client1 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.286451
Model client1 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.277406
Model client1 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.257632
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.281490
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.289867
Model client3 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.282404
Model client3 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.262814
Model client3 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.284983
Model client14 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.279608
Model client14 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.289945
Model client14 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.290384
Model client14 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.284473
Model client14 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.279534
Model client20 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.295043
Model client20 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.305198
Model client20 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.287851
Model 

Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.264850
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.247884
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.239660
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.220543
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.206028
Model client14 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.284847
Model client14 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.242342
Model client14 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.233973
Model client14 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.238930
Model client14 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.203155
Model client19 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.263583
Model client19 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.268747
Model client19 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.244327
Model client19 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.233722
Model client19 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.202303
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.259833
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.242560

Model client12 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.095042
Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.183826
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.167999
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.123038
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.006841
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.060601

Test set: Average loss for Global model: 2.0379, Accuracy: 2758/10000 (28%)

Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.012455
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.045847
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.913628
Model client18 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.020696
Model client18 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.978320
Model client4 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.070964
Model client4 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.197879
Model client4 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.987845
Model client4 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.044848
Model client4 Train Epoch: 5 [0/2560 (0%)

Model client15 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.237248
Model client15 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.924211
Model client15 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.868507
Model client7 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.127016
Model client7 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.067490
Model client7 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.173545
Model client7 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.935234
Model client7 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.904568
Model client20 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.900852
Model client20 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.075146
Model client20 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.929995
Model client20 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.053647
Model client20 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.794385
Model client12 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.924413
Model client12 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.032016
Model client12 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.857077
Model client12 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.432580
Mo

Model client15 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.973127
Model client15 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.917996
Model client15 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.913851
Model client15 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.238825
Model client1 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.901514
Model client1 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.785608
Model client1 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.959025
Model client1 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.932482
Model client1 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.931447
Model client2 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.759713
Model client2 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.993429
Model client2 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.968377
Model client2 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.975716
Model client2 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.785251
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.925681
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.873624
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.970484
Model c